## Create Data

In [ ]:
# Sample sentences for the list
sentences = [
    "The quick brown fox jumps over the lazy dog.",
    "She opened her book and started to read.",
    "Today is a sunny day.",
    "The cat sat on the mat.",
    "He loves to play soccer.",
    "I need to buy some groceries.",
    "Python programming is fun.",
    "The lake is deep and beautiful.",
    "She sings beautifully.",
    "They are planning a trip to Paris."
]

# Print the list of sentences
for sentence in sentences:
    print(sentence)

The quick brown fox jumps over the lazy dog.
She opened her book and started to read.
Today is a sunny day.
The cat sat on the mat.
He loves to play soccer.
I need to buy some groceries.
Python programming is fun.
The lake is deep and beautiful.
She sings beautifully.
They are planning a trip to Paris.


## Installation

In [ ]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


## Grab Secrets

In [ ]:
from google.colab import userdata

## Embeddings

In [ ]:
from openai import OpenAI
from typing import List

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

def list_to_nums(sentences: List[str]) -> List[List[float]]:
    """
    Converts a list of sentences into a list of numerical embeddings using OpenAI's embedding model.

    Args:
    - sentences (List[str]): A list of sentences (strings).

    Returns:
    - List[List[float]]: A list of lists of numerical embeddings.
    """

    # Initialize the list to store embeddings
    embeddings = []

    # Loop through each sentence to convert to embeddings
    for sentence in sentences:
        # Use the OpenAI API to get embeddings for the sentence

        response = client.embeddings.create(
            input=sentence,
            model="text-embedding-3-small"
        )

        embeddings.append(response.data[0].embedding)

    return embeddings

In [ ]:
%%time

# Call the function
query_database = list_to_nums(sentences)

CPU times: user 80.2 ms, sys: 3.73 ms, total: 83.9 ms
Wall time: 1.49 s


In [ ]:
prompt = "What is the meaning of life?"

In [ ]:
%%time

prompt_embed_ = list_to_nums([prompt])
print(type(prompt_embed_), len(prompt_embed_[0]))

<class 'list'> 1536
CPU times: user 6.93 ms, sys: 0 ns, total: 6.93 ms
Wall time: 219 ms


## Quantized Influence Measure (QIM)

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
from typing import Any, Dict, List, Tuple, Union


def quantize_to_kbit(arr: Union[np.ndarray, Any], k: int = 16, use_8bit = True) -> np.ndarray:
    """Converts an array to a k-bit representation by normalizing and scaling its values.

    Args:
        arr (Union[np.ndarray, Any]): The input array to be quantized.
        k (int): The number of levels to quantize to. Defaults to 16 for 4-bit quantization.
    Returns:
        np.ndarray: The quantized array with values scaled to 0 to k-1.
    """
    if not isinstance(arr, np.ndarray):  # Check if input is not a numpy array
        arr = np.array(arr)  # Convert input to a numpy array

    arr_min = arr.min()  # Calculate the minimum value in the array
    arr_max = arr.max()  # Calculate the maximum value in the array
    normalized_arr = (arr - arr_min) / (arr_max - arr_min)  # Normalize array values to [0, 1]
    return np.round(normalized_arr * (k - 1)).astype(int)  # Scale normalized values to 0-(k-1) and convert to integer


def quantized_influence(arr1: np.ndarray, arr2: np.ndarray, k: int = 16, use_dagger: bool = False) -> Tuple[float, List[float]]:
    """
    Calculates a weighted measure of influence based on quantized version of input arrays and optionally applies a transformation.

    Args:
        arr1 (np.ndarray): First input array to be quantized and analyzed.
        arr2 (np.ndarray): Second input array to be quantized and used for influence measurement.
        k (int): The quantization level, defaults to 16 for 4-bit quantization.
        use_dagger (bool): Flag to apply a transformation based on local averages, defaults to False.
    Returns:
        Tuple[float, List[float]]: A tuple containing the quantized influence measure and an optional list of transformed values based on local estimates.
    """
    # Quantize both arrays to k levels
    arr1_quantized = quantize_to_kbit(arr1, k)
    arr2_quantized = quantize_to_kbit(arr2, k)

    # Find unique quantized values in arr1
    unique_values = np.unique(arr1_quantized)

    # Compute the global average of quantized arr2
    total_samples = len(arr2_quantized)
    y_bar_global = np.mean(arr2_quantized)

    # Compute weighted local averages and normalize
    weighted_local_averages = [(np.mean(arr2_quantized[arr1_quantized == val]) - y_bar_global)**2 * len(arr2_quantized[arr1_quantized == val])**2 for val in unique_values]
    qim = np.sum(weighted_local_averages) / (total_samples * np.std(arr2_quantized))  # Calculate the quantized influence measure

    if use_dagger:
        # If use_dagger is True, compute local estimates and map them to unique quantized values
        local_estimates = [np.mean(arr2_quantized[arr1_quantized == val]) for val in unique_values]
        daggers = {unique_values[i]: v for i, v in enumerate(local_estimates)}  # Map unique values to local estimates

        def find_val_(i: int) -> float:
            """Helper function to map quantized values to their local estimates."""
            return daggers[i]

        # Apply transformation based on local estimates
        daggered_values = list(map(find_val_, arr1_quantized))
        return qim, daggered_values
    else:
        # If use_dagger is False, return the original quantized arr1 values
        daggered_values = arr1_quantized.tolist()
        return qim

In [ ]:
len(prompt_embed_[0]), len(query_database[0])

(1536, 1536)

### Test `QIM`

In [ ]:
%%time

current_qim = quantized_influence(prompt_embed_[0], query_database[0], k=16, use_dagger=False)
print(current_qim)

1.6049355403671182
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.11 ms


In [ ]:
%%time

temp = [quantized_influence(prompt_embed_[0], query_database[0], k=16, use_dagger=False) for i in range(5000)]

CPU times: user 3.61 s, sys: 11.5 ms, total: 3.62 s
Wall time: 3.63 s


In [ ]:
%%time

current_qim = quantized_influence(prompt_embed_[0], query_database[0], k=3, use_dagger=False)
print(current_qim)

0.09320953320257279
CPU times: user 2.82 ms, sys: 0 ns, total: 2.82 ms
Wall time: 2.46 ms


In [ ]:
len(query_database)

10

In [ ]:
%%time


scores = [[sentences[i], query_database[i], quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)] for i in range(len(query_database))]
print(scores)

[['The quick brown fox jumps over the lazy dog.', [-0.018420802429318428, -0.007212277036160231, 0.0036315114703029394, -0.05419674143195152, -0.022696154192090034, 0.037019215524196625, 0.029052091762423515, 0.02385062538087368, 0.01124024111777544, -0.020628252997994423, -0.013396946713328362, 0.009083535522222519, -0.022620033472776413, 0.03544608876109123, -0.0161245446652174, 0.00912159588187933, -0.011703298427164555, -0.05622658133506775, -0.030625218525528908, -0.02179541252553463, -0.01830662414431572, -0.0004828800156246871, 0.001953721512109041, -0.0014359535416588187, -0.0006371003109961748, -0.008817119523882866, 0.006485974416136742, -0.031284917145967484, 0.008506299927830696, -0.002183664357289672, 0.03344162181019783, -0.029432686045765877, -0.011310016736388206, -0.06470116227865219, -0.010992854833602905, -0.03250281885266304, -0.039886362850666046, -0.012642099522054195, -0.02211257442831993, -0.00900107342749834, 0.0008182794554159045, -0.061656404286623, 0.0345072

In [ ]:
import pandas as pd

In [ ]:
%%time

refs = pd.DataFrame(scores)
refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "qim"})

CPU times: user 1.75 ms, sys: 107 µs, total: 1.86 ms
Wall time: 4.66 ms


In [ ]:
refs = refs.sort_values(by="qim", ascending=False)

In [ ]:
refs

,sentences,query_embeddings,qim
7,The lake is deep and beautiful.,"[-0.0035871239379048347, -0.023663872852921486...",0.860040
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.432711
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.158959
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",0.120254
0,The quick brown fox jumps over the lazy dog.,"[-0.018420802429318428, -0.007212277036160231,...",0.093210
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.052678
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.036660
8,She sings beautifully.,"[0.0009488609503023326, -0.03688103333115578, ...",0.016969
2,Today is a sunny day.,"[-0.00047274649841710925, -0.04247830808162689...",0.016635
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.010064


## Query Search

In [ ]:
import pandas as pd
from typing import List

def query_search(prompt: str) -> pd.DataFrame:
    """
    Takes a text prompt and searches a predefined database by converting the prompt
    and database entries to embeddings, and then calculating a quantized influence metric.

    Args:
    - prompt (str): A text prompt to search for in the database.

    Returns:
    - pd.DataFrame: A pandas DataFrame sorted by the quantized influence metric in descending order.
                     The DataFrame contains the original sentences, their embeddings, and the computed scores.
    """
    # Convert the prompt to its numerical embedding
    prompt_embed_ = list_to_nums([prompt])

    # Calculate scores for each item in the database using the quantized influence metric
    scores = [
        [
            sentences[i],  # The sentence itself
            query_database[i],  # Embedding of the sentence
            quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)  # Score calculation
        ]
        for i in range(len(query_database))
    ]

    # Convert the list of scores into a DataFrame
    refs = pd.DataFrame(scores)
    # Rename columns for clarity
    refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "qim"})
    # Sort the DataFrame based on the 'qim' score in descending order
    refs = refs.sort_values(by="qim", ascending=False)

    return refs

In [ ]:
query_search("What is the meaning of life?")

,sentences,query_embeddings,qim
7,The lake is deep and beautiful.,"[-0.0035871239379048347, -0.023663872852921486...",0.731892
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.437797
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.161556
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",0.122665
0,The quick brown fox jumps over the lazy dog.,"[-0.018420802429318428, -0.007212277036160231,...",0.093699
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.052105
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.036736
8,She sings beautifully.,"[0.0009488609503023326, -0.03688103333115578, ...",0.017012
2,Today is a sunny day.,"[-0.00047274649841710925, -0.04247830808162689...",0.016798
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.009255


In [ ]:
query_search("The lake is dup and beauti")

,sentences,query_embeddings,qim
7,The lake is deep and beautiful.,"[-0.0035871239379048347, -0.023663872852921486...",1.446535
8,She sings beautifully.,"[0.0009488609503023326, -0.03688103333115578, ...",0.264012
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.154273
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.117649
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.089566
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.087845
0,The quick brown fox jumps over the lazy dog.,"[-0.018420802429318428, -0.007212277036160231,...",0.079604
2,Today is a sunny day.,"[-0.00047274649841710925, -0.04247830808162689...",0.053530
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.052718
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",0.014177


In [ ]:
query_search("The planning to paris is not good")

,sentences,query_embeddings,qim
9,They are planning a trip to Paris.,"[0.011352742090821266, -0.011727719567716122, ...",1.996888
0,The quick brown fox jumps over the lazy dog.,"[-0.018420802429318428, -0.007212277036160231,...",0.187576
3,The cat sat on the mat.,"[-0.024405060335993767, -0.03737139701843262, ...",0.153301
8,She sings beautifully.,"[0.0009488609503023326, -0.03688103333115578, ...",0.096750
6,Python programming is fun.,"[0.01767251081764698, -0.03974471241235733, -0...",0.046311
4,He loves to play soccer.,"[-0.0007886678795330226, -0.009807299822568893...",0.030507
7,The lake is deep and beautiful.,"[-0.0035871239379048347, -0.023663872852921486...",0.012530
5,I need to buy some groceries.,"[0.004588521085679531, -0.0148843415081501, -0...",0.009558
2,Today is a sunny day.,"[-0.00047274649841710925, -0.04247830808162689...",0.004870
1,She opened her book and started to read.,"[0.013224396854639053, 0.042468324303627014, -...",0.003036


## Read Data

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
from typing import List, Tuple
import PyPDF2

def read_and_textify(files: List[str], chunk_size: int = 30) -> Tuple[List[str], List[str]]:
    """
    Reads PDF files from given paths and extracts text from each page.

    This function iterates over a list of PDF file paths, opens each file, extracts text from each page,
    and compiles a list of texts and corresponding source information.

    Args:
    files (List[str]): A list of paths to PDF files.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing two lists:
        1. A list of strings, where each string is the text extracted from a PDF page.
        2. A list of strings indicating the source of each text (file name and page number).
    """

    text_list = []  # List to store extracted text
    sources_list = []  # List to store source information

    # Iterate over each file path
    for file_path in files:
        # Open the PDF file
        with open(file_path, "rb") as file:
            pdfReader = PyPDF2.PdfReader(file)  # Create a PDF reader object
            # Iterate over each page in the PDF
            for i in range(len(pdfReader.pages)):
                pageObj = pdfReader.pages[i]  # Get the page object
                text = pageObj.extract_text()  # Extract text from the page
                if text:
                    # Split text into chunks of approximately 'chunk_size' words
                    words = text.split('. ')
                    for j in range(0, len(words), chunk_size):
                        chunk = ". ".join(words[j : j + chunk_size]) + '.'
                        text_list.append(chunk)
                        # Create a source identifier for each chunk and add it to the list
                        sources_list.append(f"{file.name}_page_{i}_chunk_{j // chunk_size}")
                else:
                    # If no text extracted, still add a placeholder
                    text_list.append("")
                    sources_list.append(f"{file.name}_page_{i}_chunk_0")
                pageObj.clear()  # Clear the page object (optional, for memory management)

    return text_list, sources_list

In [ ]:
%%time

uploaded_files = "/content/CL Vapaad Paper by Yin.pdf"

# Process the uploaded files to extract text and source information
textify_output = read_and_textify([uploaded_files], chunk_size=2)

# Separate the output into documents (text) and their corresponding sources
documents, sources = textify_output

CPU times: user 34.4 ms, sys: 968 µs, total: 35.4 ms
Wall time: 35.9 ms


In [ ]:
type(documents), type(sources)

(list, list)

In [ ]:
documents[0]

'Dear Editor , \n \nI hope this letter finds you well. I am writing to submit our manuscript titled "Vision Augmentation Prediction Autoencoder with Attention Design (V APAAD)" for consideration as a research article in the esteemed journal of Machine Learning and Knowledge Extraction.'

In [ ]:
len(documents), len(sources)

(9, 9)

In [ ]:
documents[0], sources[0]

('Dear Editor , \n \nI hope this letter finds you well. I am writing to submit our manuscript titled "Vision Augmentation Prediction Autoencoder with Attention Design (V APAAD)" for consideration as a research article in the esteemed journal of Machine Learning and Knowledge Extraction.',
 '/content/CL Vapaad Paper by Yin.pdf_page_0_chunk_0')

In [ ]:
%%time

# Call the function
query_database = list_to_nums(documents)

CPU times: user 44.9 ms, sys: 6.8 ms, total: 51.7 ms
Wall time: 1.31 s


In [ ]:
type(query_database), len(query_database)

(list, 9)

In [ ]:
import pandas as pd
from typing import List

def query_search(prompt: str, sentences: list[str], query_database: list[list[float]], sources: list[str]) -> pd.DataFrame:
    """
    Takes a text prompt and searches a predefined database by converting the prompt
    and database entries to embeddings, and then calculating a quantized influence metric.

    Args:
    - prompt (str): A text prompt to search for in the database.

    Returns:
    - pd.DataFrame: A pandas DataFrame sorted by the quantized influence metric in descending order.
                     The DataFrame contains the original sentences, their embeddings, and the computed scores.
    """
    # Convert the prompt to its numerical embedding
    prompt_embed_ = list_to_nums([prompt])

    # Calculate scores for each item in the database using the quantized influence metric
    scores = [
        [
            sentences[i],  # The sentence itself
            query_database[i],  # Embedding of the sentence
            sources[i],  # Source of the sentence
            quantized_influence(prompt_embed_[0], query_database[i], k=3, use_dagger=False)  # Score calculation
        ]
        for i in range(len(query_database))
    ]

    # Convert the list of scores into a DataFrame
    refs = pd.DataFrame(scores)
    # Rename columns for clarity
    refs = refs.rename(columns={0: "sentences", 1: "query_embeddings", 2: "page no", 3: "qim"})
    # Sort the DataFrame based on the 'qim' score in descending order
    refs = refs.sort_values(by="qim", ascending=False)

    return refs

In [ ]:
%%time

prompt = "what is vapaad?"
ref_tab = query_search(prompt, documents, query_database, sources)
ref_tab

CPU times: user 22 ms, sys: 894 µs, total: 22.9 ms
Wall time: 326 ms


,sentences,query_embeddings,page no,qim
3,"V APAAD combines data augmentation, ConvLSTM2D...","[0.004169906955212355, -0.003684121882542968, ...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.560549
4,The experimental results confirm that V APAAD ...,"[0.017516417428851128, 0.013900701887905598, 0...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.369587
2,This study introduces the Vision Augmentation ...,"[0.029335709288716316, 0.009453258477151394, -...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.304369
0,"Dear Editor , \n \nI hope this letter finds yo...","[0.05714760720729828, -0.013315004296600819, -...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.199235
1,Our \nwork introduces an innovative autoencode...,"[0.04408750310540199, 0.013028436340391636, -0...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.074417
5,All authors have approved the manuscript and a...,"[0.04740281403064728, 0.0023992909118533134, 0...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.037588
7,Thank you for considering our work for public...,"[0.024561084806919098, -0.014052295126020908, ...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.021880
8,"\n Warm regards, \nYiqiao Yin \nAffiliation:...","[-0.0027448309119790792, -0.03821829333901405,...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.021309
6,"\n Attached to this email, you will find our ...","[0.044033411890268326, 0.016705965623259544, 0...",/content/CL Vapaad Paper by Yin.pdf_page_0_chu...,0.012187


In [ ]:
ref_tab.to_json()

'{"sentences":{"12":"or documentation. Warm regards, Yiqiao Yin Affiliation: Columbia University, University of Chicago","11":"the possibility of contributing to your journal. Thank you for considering our work for publication. Please do not hesitate to contact me at [yy2502@columbia.edu] should you need any additional information","6":"enhancing predictive accuracy and context -aware analysis. This methodology not only adheres to human cognitive processes during video interpretation but also addresses limitations in conventional models, which often struggle with","9":"Learning and Knowledge Extraction. We believe that our manuscript would be a significant contribution to the field, aligning well with the journal\'s focus on innovative applications of machine learning. Attached","7":"the variability inherent in video sequences. The experimental results confirm that V APAAD outperforms existing models, especially in integrating attention mechanisms, which significantly improve predictiv

## Call GPT

In [ ]:
def call_gpt(prompt: str, content: str) -> str:
    """
    Sends a structured conversation context including a system prompt, user prompt,
    and additional background content to the GPT-3.5-turbo model for a response.

    This function is responsible for generating an AI-powered response by interacting
    with the OpenAI API. It puts together a preset system message, a formatted user query,
    and additional background information before requesting the completion from the model.

    Args:
        prompt (str): The main question or topic that the user wants to address.
        content (str): Additional background information or details relevant to the prompt.

    Returns:
        str: The generated response from the GPT model based on the given prompts and content.

    Note: 'client' is assumed to be an already created and authenticated instance of the OpenAI
          client, which should be set up prior to calling this function.
    """

    # Generates a response from the model based on the interactive messages provided
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # The AI model being queried for a response
        messages=[
            # System message defining the assistant's role
            {"role": "system", "content": "You are a helpful assistant."},
            # User message containing the prompt
            {"role": "user", "content": f"I want to ask you a question: {prompt}"},
            # Assistant message asking for background content
            {"role": "assistant", "content": "What is the background content?"},
            # User providing the background content
            {"role": "user", "content": content},
        ]
    )

    # Extracts and returns the response content from the model's completion
    return response.choices[0].message.content

In [ ]:
%%time

call_gpt(prompt, ' '.join(list(ref_tab.sentences)))

CPU times: user 35.9 ms, sys: 8.97 ms, total: 44.8 ms
Wall time: 3.48 s


'V APAAD is an innovative approach called Vision Augmentation Prediction Autoencoder with Attention Design that integrates attention mechanisms into sequence prediction, particularly enhancing the understanding of temporal dynamics in video sequences. The methodology combines data augmentation, ConvLSTM2D layers, and a custom-built self-attention mechanism to focus on salient features within a sequence, thus improving predictive accuracy and context-aware analysis. The experimental results suggest that V APAAD outperforms existing models, especially in integrating attention mechanisms which enhance predictive performance in video interpretation. This approach demonstrates robust performance and superior handling of complex temporal data compared to traditional methods, as shown through experiments conducted using the Moving MNIST dataset.'

In [ ]:
prompt

'what is vapaad?'